In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

## Chargement des données

Les données sont chargées en morceaux à l'aide de pd.read_csv avec chunksize=5000000, ce qui est une bonne pratique pour gérer de grands fichiers sans surcharger la mémoire​​.

In [3]:
file_path = "bioconcepts2pubtatorcentral"
chunk_iterator = pd.read_csv(file_path, header=None, delimiter="\t", chunksize=5000000)
data_chunk = next(chunk_iterator)
data_chunk

,0,1,2,3,4
0,3958000,Species,9606,patients,SR4GN
1,3958000,Disease,MESH:D012544,Scheuermann's kyphosis,TaggerOne
2,3958000,Disease,MESH:D007738,kyphosis,TaggerOne|MESH
3,3958000,Disease,MESH:D011183,NaN,MESH
4,23574000,Species,9615,dogs,SR4GN
...,...,...,...,...,...
4999995,7675011,Chemical,MESH:D009569,nitric oxide,TaggerOne|MESH
4999996,7675011,Disease,MESH:D001925,brain damage,TaggerOne
4999997,7675011,Disease,MESH:D064420,cytotoxicity,TaggerOne
4999998,7675011,Chemical,MESH:D000438,Alcohol|alcohol,TaggerOne


## Prétraitement

Les données sont nettoyées en remplaçant les valeurs NaN par des chaînes vides et en filtrant les lignes pour ne conserver que celles avec des concepts MeSH (identifiés par MESH:). Ensuite, les concepts MeSH sont regroupés par document (ou un autre identifiant), et chaque groupe de concepts est converti en une chaîne de caractères séparés par des virgules​

In [4]:
data_chunk = data_chunk.fillna('')

mesh_data = data_chunk[data_chunk[2].str.startswith('MESH:')]

grouped = mesh_data.groupby(0)[2].apply(list)

mesh_list_df = grouped.apply(lambda x: ','.join(x)).reset_index(drop=True)

mesh_list_df.to_csv("./JClose/mesh/mesh_data_processed.csv", index=False, header=False)

grouped

0
1                  [MESH:D000432, MESH:D005561, MESH:D002245]
2                  [MESH:C562386, MESH:D009243, MESH:D009249]
3           [MESH:D003643, MESH:D008670, MESH:D019966, MES...
4                  [MESH:D002738, MESH:D003911, MESH:D007966]
6                  [MESH:D010100, MESH:D003035, MESH:D007501]
                                  ...                        
38060005    [MESH:D003643, MESH:D010100, MESH:D001327, MES...
38060006                                       [MESH:D000152]
38060008    [MESH:C030601, MESH:D010718, MESH:C085678, MES...
38060009                                       [MESH:D009369]
38060010           [MESH:D006695, MESH:D010757, MESH:D007669]
Name: 2, Length: 311235, dtype: object

## Transformation des transactions

Utilisation de ***TransactionEncoder*** de la bibliothèque mlxtend pour transformer les transactions en un format approprié pour l'analyse des fréquences des itemsets. Cela prépare les données pour l'application de l'algorithme Apriori​

In [5]:
# Convertir la série 'grouped' en une liste de listes
transactions = grouped.tolist()

# Utilisation de TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

df

,MESH:C000006,MESH:C000009,MESH:C000015,MESH:C000020,MESH:C000027,MESH:C000028,MESH:C000050,MESH:C000061,MESH:C000062,MESH:C000078,...,MESH:D066190,MESH:D066229,MESH:D066247,MESH:D066248,MESH:D066253,MESH:D066255,MESH:D066257,MESH:D066258,MESH:D066259,MESH:D066263
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311230,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
311231,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
311232,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
311233,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Application de l'algorithme Apriori

L'algorithme Apriori est appliqué avec un seuil de support minimal de 0,1 pour identifier les itemsets fréquents, suivi de la génération des règles d'association avec un seuil de confiance minimal de 0,01. Cette étape produit les règles d'association entre concepts MeSH qui satisfont les critères de support et de confiance spécifiés​

In [6]:
# Application de l'algorithme Apriori
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.024698,(MESH:C000657245)
1,0.011217,(MESH:C003043)
2,0.012592,(MESH:D000014)
3,0.012801,(MESH:D000069285)
4,0.029068,(MESH:D000079225)
...,...,...
260,0.011846,"(MESH:D003643, MESH:D003920, MESH:D006973)"
261,0.011178,"(MESH:D003643, MESH:D003920, MESH:D009369)"
262,0.011699,"(MESH:D003643, MESH:D007239, MESH:D009369)"
263,0.011878,"(MESH:D003643, MESH:D007249, MESH:D009369)"


In [9]:
# Génération des règles d'association
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'conviction']]

,antecedents,consequents,support,confidence,lift,conviction
0,(MESH:C000657245),(MESH:D003643),0.012418,0.502797,4.600288,1.791427
1,(MESH:D000230),(MESH:D009369),0.010410,0.792758,5.254800,4.097309
2,(MESH:D001007),(MESH:D000275),0.012177,0.652211,16.610015,2.762407
3,(MESH:D000432),(MESH:D014867),0.010131,0.639943,7.746897,2.547913
4,(MESH:D001102),(MESH:D007239),0.010185,0.710603,7.916262,3.145282
5,(MESH:D001943),(MESH:D009369),0.027728,0.788848,5.228888,4.021452
6,(MESH:D002294),(MESH:D009369),0.010744,0.862300,5.715764,6.166576
7,(MESH:D002318),(MESH:D003920),0.017511,0.521332,9.078309,1.969158
8,(MESH:D003141),(MESH:D007239),0.012916,0.563578,6.278371,2.085675
9,(MESH:D003924),(MESH:D003920),0.012852,0.843348,14.685808,6.016996
